[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/prokaj/stat-learning-2022/blob/main/week1/Lab_Ch3-Linear_Regression_in_Python.ipynb)
    

This lab on Linear Regression is a python adaptation of p. 110-120 of "Introduction to Statistical Learning with Applications in R" by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani. 
It is based on the work of R. Jordan Crouser at Smith College for SDS293: Machine Learning (Spring 2016).

# Warmup exercises


## Problem 1.

Suppose we have a data set with five predictors, $$\begin{aligned}
    X_1 &= \text{GPA},\\
    X_2 &= \text{IQ},\\
    X_3 &= \text{Level},\quad \text{1 for College and 0 for High
          School},\\
    X_4 &= \text{Interaction between GPA and IQ}, \\
    X_5 &= \text{Interaction between GPA and Level}. 
  \end{aligned}$$ The response is starting salary after graduation (in
thousands of dollars). Suppose we use least squares to fit the model,
and get $\hat\beta_0 = 50$, $\hat \beta_1 = 20$, $\hat\beta_2 = 0.07$,
$\hat\beta_3 = 35$, $\hat\beta_4 = 0.01$, $\hat\beta_5 = -10$.

a. Which answer is correct, and why?

   -   For a fixed value of IQ and GPA, high school graduates earn
        more, on average, than college graduates.

   -   For a fixed value of IQ and GPA, college graduates earn more, on
        average, than high school graduates.

   -   For a fixed value of IQ and GPA, high school graduates earn
        more, on average, than college graduates provided that the GPA
        is high enough.

   -   For a fixed value of IQ and GPA, college graduates earn more, on
        average, than high school graduates provided that the GPA is
        high enough.

    

b.  Predict the salary of a college graduate with IQ of 110 and a GPA of
    4.0.


In [ ]:
beta = np.array([50, 20 , 0.07, 35, 0.01, -10])


c.  True or false: Since the coefficient for the GPA/IQ interaction term
    is very small, there is very little evidence of an interaction
    effect. Justify your answer.

## Problem 2.

This exercise relates to the College data set, which can be found in the
file [`College.csv`](https://www.statlearning.com/s/College.csv) on the
ISL book website. It contains a number of variables for 777 different
universities and colleges in the US. The variables are

-   `Private` : Public/private indicator

-   `Apps` : Number of applications received

-   `Accept` : Number of applicants accepted

-   `Enroll` : Number of students enrolled

-   `Top10perc` : New students from top 10 % of high school class

-   `Top25perc` : New students from top 25 % of high school class

-   `F.Undergrad` : Number of full-time undergraduates

-   `P.Undergrad` : Number of part-time undergraduates

-   `Outstate` : Out-of-state tuition

-   `Room.Board` : Room and board costs

-   `Books` : Estimated book costs

-   `Personal` : Estimated personal spending

-   `PhD` : Percent of faculty with Ph.D.'s

-   `Terminal` : Percent of faculty with terminal degree

-   `S.F.Ratio` : Student/faculty ratio

-   `perc.alumni` : Percent of alumni who donate

-   `Expend` : Instructional expenditure per student

-   `Grad.Rate` : Graduation rate

a.  Use the `pd.read_csv` function to load the data. Call the loaded
    data `college`, for example

    


In [ ]:
import pandas as pd

url = "https://www.statlearning.com/s/College.csv"

college = pd.read_csv(url, index_col=0)

In [ ]:
import statsmodels.datasets as smd

col = smd.get_rdataset('College', 'ISLR')


In [ ]:
col?


In [ ]:
col.data, col.package, col.title

b.  Look at the data using the `head` method. You should notice that the
    row index is just the name of each university. This is the effect of
    `index_col=0`.

In [ ]:
 college.head()

c.  Use the `describe` function to produce a numerical summary of the
    variables in the data set.


In [ ]:
    college.describe()

d. Use the `pair_plot` function from the `seaborn` package to produce a
   scatterplot matrix of the first ten columns or variables of the
   data. Recall that you can reference the first ten columns of a
   `pandas.DataFrame` by applying the `.iloc(1)[:10]` method on it.

   Note that `Private` is missing from the plot. Why?

    

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

smalldf = college.iloc(1)[:10]
smalldf['Private'] = 1*(smalldf.Private=='Yes')
sns.pairplot(smalldf)

e.  Use the `seaborn.boxplot` function to produce side-by-side boxplots
    of `Outstate` versus `Private`.


In [ ]:
sns.boxplot(x='Private', y='Outstate', data=college) 

f. Create a new qualitative variable, called `Elite`, by binning the
   `Top10perc` variable. We are going to divide universities into two
   groups based on whether or not the proportion of students coming
   from the top 10 % of their high school classes exceeds 50 %.

        college["Elite"] = college.Top10perc>50

   Use the `.value_counts` method to see how many elite universities
   there are. Now use the `boxplot()` function from the `seaborn`
   package to produce side-by-side boxplots of `Outstate` versus
   `Elite`.

    

In [ ]:
 college["Elite"] = college.Top10perc>50
 print(college.Elite.value_counts())
 sns.boxplot(x='Elite', y='Outstate', data=college)
 plt.show()


g.  Use the `.hist` method to produce some histograms with differing
    numbers of bins for a few of the quantitative variables.


In [ ]:
college.hist('Accept', bins=51)
college.hist('Room.Board', bins=21)
college[college.Private!='Yes'].hist('Outstate')

h.  Continue exploring the data, and provide a brief summary of what you
    discover.


## Problem 3.

This exercise involves the
[`Auto`](https://www.statlearning.com/s/Auto.csv) data set. Make sure
that the missing values have been removed from the data, e.g.

    #----------------
    import statsmodels.datasets as sm_datasets
    auto = sm_datasets.get_rdataset('Auto', 'ISLR')
    auto.data
    #----------------

or from the web page

    #----------------
    from urllib.request import urlopen
    import pandas as pd
    url = 'https://www.statlearning.com/s/Auto.csv'
    auto = pd.read_csv(url, na_values='?').dropna()
    auto

a.  Which of the predictors are quantitative, and which are qualitative?


In [ ]:
import statsmodels.datasets as sm_datasets
auto = sm_datasets.get_rdataset('Auto', 'ISLR')
# auto.data = auto.data.set_index('name').copy()
auto.data.head()

In [ ]:
auto?
auto.data.describe()

In [ ]:
import numpy as np

In [ ]:
disp = auto.data.displacement*(2.5**3)
disp.min(), disp.max(), disp.mean(), disp.std(), np.percentile(disp, [25, 50, 75])

b.  What is the range (minimum, maximum) of each quantitative predictor?
    You can answer this using the `min`, `max` functions or write a
    little function, called `range`, which returns this tuple.


In [ ]:
def minmax(x, axis=None):
    return np.min(x, axis=axis), np.max(x, axis=axis)

In [ ]:
minmax(auto.data, axis=0)

c.  What is the mean and standard deviation of each quantitative
    predictor?


In [ ]:
print(auto.data.columns)
quant = ['mpg', 'displacement', 'horsepower', 'weight', 'acceleration', 'year']
{k: (auto.data[k].mean(), auto.data[k].std()) for k in quant}

d.  Now remove the 10th through 85th observations. What is the range,
    mean, and standard deviation of each predictor in the subset of the
    data that remains?


In [ ]:
import matplotlib.pyplot as plt


In [ ]:
idx = np.arange(len(auto.data))
idx = (idx<10)+(idx>85)
auto1 = auto.data[idx]
plt.plot(auto1[quant].mean(axis=0),'o')
plt.plot(auto.data[quant].mean(axis=0),'o',alpha=0.5)
plt.title('mean')
plt.show()

plt.plot(auto1[quant].min(axis=0),'o')
plt.plot(auto.data[quant].min(axis=0),'o',alpha=0.5)
plt.title('min')
plt.show()


e.  Using the full data set, investigate the predictors graphically,
    using scatterplots or other tools of your choice. Create some plots
    highlighting the relationships among the predictors. Comment on your
    findings.


In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(auto.data[quant])

In [ ]:
import pandas as pd

In [ ]:
cyl_yr = auto.data.groupby(['cylinders', 'year']).size()
col_yr_cnt = cyl_yr.values
col_yr = np.array(list(cyl_yr.index))
col_yr = pd.DataFrame(col_yr, columns=['cylinders', 'year'])
col_yr['count'] = col_yr_cnt
sns.scatterplot(x='year', y='cylinders', size='count', 
                hue='count', sizes=(10, 250), legend=False, data=col_yr)

f.  Suppose that we wish to predict gas mileage (`mpg`) on the basis of
    the other variables. Do your plots suggest that any of the other
    variables might be useful in predicting `mpg`? Justify your answer.


In [ ]:
plt.figure(figsize=(10, 10))
sns.heatmap(auto.data.drop(columns=['cylinders', 'origin']).corr(), annot=True, vmin=-1, vmax=1)

In [ ]:
print(auto.data.cylinders.value_counts())
print(auto.data.origin.value_counts())



In [ ]:
for level in auto.data.cylinders.unique():
    if level!=4:
        auto.data[f'cyl_{level}'] = 1*(auto.data.cylinders==level)

for level in auto.data.origin.unique():
    if level!=1:
        auto.data[f'orig_{level}'] = 1*(auto.data.origin==level)


In [ ]:
plt.figure(figsize=(10, 10))
sns.heatmap(auto.data.drop(columns=['cylinders', 'origin']).corr(), 
            annot=True, vmin=-1, vmax=1,
            cmap = sns.diverging_palette(220, 20, n=50))


# Section 3.6 from ISL
## Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# Tells matplotlib to display images inline instead of a new window
%matplotlib inline
import statsmodels.api as sm

We'll start by importing the data from [Boston.csv](http://www.science.smith.edu/~jcrouser/SDS293/data/Boston.csv) into a pandas dataframe:

In [ ]:
from_smith_edu = False # or True 

if not from_smith_edu:
    from sklearn.datasets import load_boston
    ds = load_boston()
    colnames = [line.split()[1].lower() for line in ds['DESCR'].split('\n') if "    -" in line]
    vars = np.concatenate([ds.data, ds.target[:,None]], axis=-1)
    df = pd.DataFrame(vars, columns=colnames)
    df.__doc__=ds['DESCR']
else:
    ## alternatively we could use this:
    url = 'https://www.science.smith.edu/~jcrouser/SDS293/data/Boston.csv'
    df = pd.read_csv(url, index_col=0)

print(df.head())
print(df.describe())

In [ ]:
print(df.__doc__)

## 3.6.2 Simple Linear Regression

Now let's fit a simple linear model (`OLS` - for "ordinary least squares" method) with `medv` as the response and `lstat` as the predictor:

In [ ]:
lm = sm.OLS.from_formula('medv ~ lstat', df)
result = lm.fit()

To get detailed information about the model, we can print the results of a call to the `.summary()` method:

In [ ]:
print(result.summary().as_text())

Want individual attributes? You can access them independently like this:

In [ ]:
result.rsquared, result.fvalue, result.params.Intercept, result.params.lstat

For a complete list of attributes and methods of a `RegressionResults` object, see: http://statsmodels.sourceforge.net/devel/generated/statsmodels.regression.linear_model.RegressionResults.html?highlight=regressionresults

For quick help use:

In [ ]:
result?

Now let's try making some predictions using this model. First, we'll set up a dataframe containing the `lstat` values for which we want to predict a response:

In [ ]:
new = pd.DataFrame([[1, 5], [1, 10], [1, 15]], columns=['Intercept', 'lstat'])

Now we just call the `.predict()` method:

In [ ]:
result.predict(new)

Technically those are the right prediction values, but maybe it would be good to have the confidence intervals along with them. 

Recall that, when the linear model describes the response correctly
$$
    \hat\beta = (X^TX)^{-1}X^Ty = (X^TX)^{-1}X^T (X \beta+\varepsilon)=\beta + (X^TX)^{-1}X^T \varepsilon\sim N(\beta, \sigma^2(X^TX)^{-1}) 
$$

When we apply the model at a new observation $x$, then
$$
    \hat{y} = x\hat\beta = x\beta + x(\hat\beta-\beta)
$$
Here $x\beta$ is true mean of the unobserved response $y$ that belongs to $x$.
Therefore
$$
    y = x\beta+\varepsilon' = \hat y + x(\beta-\hat\beta) + \varepsilon' 
$$
Here $\beta-\hat\beta$ and $\varepsilon$ are independent noise so overall the variance of $y-\hat y$ is 
$x\Sigma(\hat\beta)x^T+\sigma^2 = \sigma^2(x(X^TX)^{-1}x^T+1)$ 
$\sigma^2$ is unknown it is estimated from the data.

$$
\frac{y-\hat y}{\hat\sigma \sqrt{1+x(X^TX)^{-1}x^T+1}}=\frac{y-\hat y}{\hat\sigma_0}\sim t_{n-(p+1)}
$$

From this
$$
    \mathbb{P}(y \in (\hat y-(t_\alpha/2)\hat\sigma_0, \hat y+(t_\alpha/2)\hat\sigma_0))=\alpha,\quad \text{where $t_{\alpha/2}$ is the critical value.} 
$$

In [ ]:
Sigma=result.cov_params()
(new.dot(Sigma)*new).sum(axis=1)

In [ ]:
result.cov_params()

The covariance of the `intercept` term and the `lstat` coefficient is negative. Student-t distribution is in the `scipy.stats` module.
Let's write a little helper function to get that and package it all up:

In [ ]:
from  scipy.stats import t as student_t
def predict(res, new, alpha=0.05):
    fit = pd.DataFrame(res.predict(new), columns=['fit'])
    # the variance of the predicted value is <new Sigma, new> (new is a row vector).
    sigma = (new.dot(res.cov_params())*new).sum(axis=1)**0.5
    # isf is the inverse survival function, 
    # that is the inverse of 1-F, where F is the distribution function
    t = student_t(df=res.df_resid, scale=sigma).isf(alpha/2)
    fit['lower'] = fit.fit-t
    fit['upper'] = fit.fit+t
    return fit
    

This is a confidence in the expected value of the response (the value of the regression function). Since new observations at these levels of `lstat` also contains the noise term, the confidence intervals for the new observed response values are much higher. 

In [ ]:
def predict(res, new, interval='confidence', alpha=0.05):
    fit = pd.DataFrame(res.predict(new), columns=['fit'])
    sigma2 = (new.dot(res.cov_params())*new).sum(axis=1)
    if interval=='prediction':
        sigma2 += res.scale
    sigma = sigma2**.5
    t = student_t(df=res.df_resid, scale=sigma).isf(alpha/2)
    fit['lower'] = fit.fit-t
    fit['upper'] = fit.fit+t
    return fit

In [ ]:
print(predict(result, new)) 
print(predict(result, new, interval='prediction')) 

Seaborn is a Python visualization library based on matplotlib that provides a high-level interface for drawing attractive statistical graphics.

In [ ]:
import seaborn as sns

We will now plot `medv` and `lstat` along with the least squares regression line using the `regplot()` function. We can define the color of the fit line using `line_kws` ("line keywords"):

In [ ]:
sns.regplot(x='lstat', y='medv', data=df, line_kws = {"color": "r"}, ci=None)
plt.grid()
plt.show()

We can also plot the residuals against the fitted values:

In [ ]:
result_df = pd.DataFrame({'Fitted values': result.fittedvalues, 'Residuals': result.resid})
result_df

In [ ]:
sns.regplot(x='Fitted values', y='Residuals', data=result_df, fit_reg=False)
plt.grid()
plt.show()

Perhaps we want studentized residuals instead? (This is almost the same as the residuals normalized to have unit norm, the latter is available as `.resid_pearson`)

In [ ]:
sns.scatterplot(x=np.arange(result.nobs), 
                y=result.get_influence().resid_studentized-result.resid_pearson)
plt.grid()
plt.show()

In [ ]:
result_df['S. Residuals'] = result.get_influence().resid_studentized
sns.regplot(x='Fitted values', y='S. Residuals', data=result_df,  fit_reg=False)
plt.grid()
plt.show()

We can also look for points with high leverage:

In [ ]:
from statsmodels.stats.outliers_influence import OLSInfluence
result_df['Leverage'] = OLSInfluence(result).hat_matrix_diag
sns.regplot(x='Leverage', y='S. Residuals', data=result_df,  fit_reg=False)
plt.grid()
plt.show()

## Section 3.6.3 Multiple Linear Regression

In order to fit a multiple linear regression model using least squares, we again use the `from_formula()` function. The syntax `from_formula(y∼x1+x2+x3)` is used to fit a model with three predictors, `x1`, `x2`, and `x3`. The `summary()` function now outputs the regression coefficients for all the predictors.

In [ ]:
model = sm.OLS.from_formula('medv ~ lstat + age', df)
result = model.fit()
print(result.summary())

The Boston data set contains 13 variables, and so it would be cumbersome to have to type all of these in order to perform a regression using all of the predictors. Instead, we can use the following short-hand:

In [ ]:
# All columns (except medv, which is our response)
model = sm.OLS.from_formula('medv ~ ' + '+'.join(df.columns.difference(['medv'])), df)
result = model.fit()
print(result.summary())

Note that we used the syntax `.join(df.columns.difference(['medv']))` to exclude the response variable above. We can use the same napproach to perform a regression using just a subset of the predictors? For example, in the above regression output, `age` and `indus` have a high p-value. So we may wish to run a regression excluding these predictors:

In [ ]:
# All columns (except medv age and indus)
model = sm.OLS.from_formula('medv ~ ' + '+'.join(df.columns.difference(['medv', 'age', 'indus'])), df)
result = model.fit()
print(result.summary())

## Section 3.6.4 Interaction Terms

It is easy to include interaction terms in a linear model using the `.from_formula()` function. The syntax `lstat:black` tells Python to include an interaction term between `lstat` and `black`. The syntax `lstat*age` simultaneously includes `lstat`, `age`, and the interaction term `lstat*age` as predictors; it is a shorthand for `lstat+age+lstat:age`.

In [ ]:
print(sm.OLS.from_formula('medv ~ lstat*age', df).fit().summary())

## Section 3.6.5 Non-linear Transformations of the Predictors

The `.from_formula()` function can also accommodate non-linear transformations of the predictors. For instance, given a predictor `X`, we can create a predictor `X^2` using `np.square(X)`. We now perform a regression of `medv` onto `lstat` and `lstat^2`.

In [ ]:
lm.fit2 = sm.OLS.from_formula('medv ~ lstat + np.square(lstat)', df).fit()
print(lm.fit2.summary())

The near-zero p-value associated with the quadratic term suggests that it leads to an improved model. We use the `anova_lm()` function to further quantify the extent to which the quadratic fit is superior to the linear fit.

In [ ]:
lm.fit = sm.OLS.from_formula('medv ~ lstat', df).fit()
print(sm.stats.anova_lm(lm.fit, lm.fit2))

Here Model 0 represents the linear submodel containing only one predictor, `lstat`, while Model 1 corresponds to the larger quadraticmodel that has two predictors, `lstat` and `lstat2`. The `anova_lm()` function performs a hypothesis test comparing the two models. The null hypothesis is that the two models fit the data equally well, and the alternative hypothesis is that the full model is superior. 

The F-statistic is 135 and the associated p-value is virtually zero. This provides very clear evidence that the model containing the predictors `lstat` and `lstat2` is far superior to the model that only contains the predictor `lstat`. This is not surprising, since earlier we saw evidence for non-linearity in the relationship between `medv` and `lstat`. 

If we type:

In [ ]:
fitted_values = pd.Series(lm.fit2.fittedvalues, name="Fitted Values")
residuals = pd.Series(lm.fit2.get_influence().resid_studentized, name="S. Residuals")
sns.regplot(x=fitted_values, y=s_residuals,  fit_reg=False)
plt.grid()
plt.show()

then we see that when the `lstat2` term is included in the model, there is little discernible pattern in the residuals.

In order to create a cubic fit, we can include a predictor of the form `np.power(x, 3))`. However, this approach can start to get cumbersome for higher order polynomials. A better approach involves using list comprehension inside a `.join()`. For example, the following command produces a fifth-order polynomial fit:

In [ ]:
formula_5 = 'medv ~ ' + '+'.join(['np.power(lstat,' + str(i) + ')' for i in range(1,6)])
print(formula_5)
print(sm.OLS.from_formula(formula_5, df).fit().summary())

Of course, we are in no way restricted to using polynomial transformations of the predictors. Here we try a log transformation.

In [ ]:
print(sm.OLS.from_formula('medv ~ np.log(rm)', df).fit().summary())

## Section 3.6.6 Qualitative Predictors

We will now examine the [`Carseats`](http://www.science.smith.edu/~jcrouser/SDS293/data/Carseats.csv) data that we talked about earlier in class. We will attempt to predict `Sales` (child car seat sales) in 400 locations based on a number of predictors.

In [ ]:
if from_smith_edu:
    df2 = pd.read_csv('Carseats.csv')
else:
    import statsmodels.datasets as smd
    df2 = smd.get_rdataset('Carseats', 'ISLR')
    df2.data.__doc__= df2.__doc__
    df2 = df2.data
df2.head()

In [ ]:
print(df2.__doc__)

The `Carseats` data includes qualitative predictors such as `Shelveloc`, an indicator of the quality of the shelving location—that is, the space within a store in which the car seat is displayed—at each location. The predictor `Shelveloc` takes on three possible values, `Bad`, `Medium`, and `Good`.

Given a qualitative variable such as `Shelveloc`, Python generates dummy variables automatically. Below we fit a multiple regression model that includes some interaction terms.

In [ ]:
formula_carseat = 'Sales ~ Income:Advertising+Price:Age + ' + " + ".join(df2.columns.difference(['Sales']))
print(formula_carseat)
print(sm.OLS.from_formula(formula_carseat, df2).fit().summary())

To learn how to set other coding schemes (or _contrasts_), see: http://statsmodels.sourceforge.net/devel/examples/notebooks/generated/contrasts.html

In [46]:
import scipy.stats

In [49]:
scipy.stats.normaltest??

Signature: scipy.stats.normaltest(a, axis=0, nan_policy='propagate')
Source:   
def normaltest(a, axis=0, nan_policy='propagate'):
    """Test whether a sample differs from a normal distribution.

    This function tests the null hypothesis that a sample comes
    from a normal distribution.  It is based on D'Agostino and
    Pearson's [1]_, [2]_ test that combines skew and kurtosis to
    produce an omnibus test of normality.

    Parameters
    ----------
    a : array_like
        The array containing the sample to be tested.
    axis : int or None, optional
        Axis along which to compute test. Default is 0. If None,
        compute over the whole array `a`.
    nan_policy : {'propagate', 'raise', 'omit'}, optional
        Defines how to handle when input contains nan.
        The following options are available (default is 'propagate'):

          * 'propagate': returns nan
          * 'raise': throws an error
          * 'omit': performs the calculations ignoring nan values

